In [0]:
# https://keras.io/
!pip install -q keras
import keras

In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation ,GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import scipy 
import numpy as np


In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 100
num_filter = 12
compression = 0.5
dropout_rate = 0.2
val_batch_size=60

In [27]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



x_train_resized=np.array([scipy.misc.imresize(img,(24,24)) for img in x_train])



/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:482: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if issubdtype(ts, int):
/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:485: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif issubdtype(type(size), float):


In [28]:
x_test_resized=np.array([scipy.misc.imresize(img,(24,24)) for img in x_test])
x_train_resized[1].shape


/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:482: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if issubdtype(ts, int):
/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:485: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif issubdtype(type(size), float):


(24, 24, 3)

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_1_1=Conv2D(num_classes*10,(1,1))(relu)

    AvgPooling = GlobalAveragePooling2D()(Conv2D_1_1)
#     flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(AvgPooling)
    
    return output

In [0]:
num_filter = 12
dropout_rate = 0
l = 20
input = Input(shape=(None,None , channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [48]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_753 (Conv2D)             (None, None, None, 1 324         input_10[0][0]                   
__________________________________________________________________________________________________
batch_normalization_749 (BatchN (None, None, None, 1 48          conv2d_753[0][0]                 
__________________________________________________________________________________________________
activation_748 (Activation)     (None, None, None, 1 0           batch_normalization_749[0][0]    
__________________________________________________________________________________________________
conv2d_754

batch_normalization_815 (BatchN (None, None, None, 2 96          concatenate_775[0][0]            
__________________________________________________________________________________________________
activation_814 (Activation)     (None, None, None, 2 0           batch_normalization_815[0][0]    
__________________________________________________________________________________________________
conv2d_820 (Conv2D)             (None, None, None, 6 1296        activation_814[0][0]             
__________________________________________________________________________________________________
concatenate_776 (Concatenate)   (None, None, None, 3 0           concatenate_775[0][0]            
                                                                 conv2d_820[0][0]                 
__________________________________________________________________________________________________
batch_normalization_816 (BatchN (None, None, None, 3 120         concatenate_776[0][0]            
__________

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.SGD(lr=0.1),
              metrics=['accuracy'])


In [50]:
# model.fit(x_train, y_train,
#                     batch_size=batch_size,
#                     epochs=epochs,
#                     verbose=1,
#                     validation_data=(x_test, y_test))
train_datagen=ImageDataGenerator(rescale=1./255,
shear_range=0.2,
zoom_range=0.2,horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)
model.fit_generator(train_datagen.flow(x_train_resized, y_train, batch_size=batch_size),
                    samples_per_epoch=len(x_train_resized) , epochs=10,
                   validation_data=val_datagen.flow(x_test_resized,y_test,batch_size=val_batch_size)
                   ,verbose=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=10, validation_data=<keras.pre..., verbose=1, steps_per_epoch=390)`
  


Epoch 1/10
390/390 [==============================] - 227s 582ms/step - loss: 1.7082 - acc: 0.3617 - val_loss: 1.5921 - val_acc: 0.4131
Epoch 2/10
280/390 [====================>.........] - ETA: 43s - loss: 1.4367 - acc: 0.4725

390/390 [==============================] - 167s 428ms/step - loss: 1.4176 - acc: 0.4807 - val_loss: 1.5709 - val_acc: 0.4471
Epoch 3/10
390/390 [==============================] - 166s 426ms/step - loss: 1.2770 - acc: 0.5360 - val_loss: 1.7535 - val_acc: 0.4346
Epoch 4/10
 17/390 [>.............................] - ETA: 2:32 - loss: 1.1983 - acc: 0.5685

390/390 [==============================] - 167s 427ms/step - loss: 1.1765 - acc: 0.5764 - val_loss: 1.4911 - val_acc: 0.4822
Epoch 5/10
321/390 [=======================>......] - ETA: 27s - loss: 1.1025 - acc: 0.6065

390/390 [==============================] - 166s 425ms/step - loss: 1.0962 - acc: 0.6083 - val_loss: 1.3686 - val_acc: 0.5274
Epoch 6/10
390/390 [==============================] - 166s 426ms/step - loss: 1.0272 - acc: 0.6343 - val_loss: 1.1751 - val_acc: 0.5860
Epoch 7/10
 26/390 [=>............................] - ETA: 2:24 - loss: 0.9917 - acc: 0.6514

390/390 [==============================] - 166s 424ms/step - loss: 0.9664 - acc: 0.6551 - val_loss: 1.5432 - val_acc: 0.5194
Epoch 8/10
324/390 [=======================>......] - ETA: 26s - loss: 0.9239 - acc: 0.6712

390/390 [==============================] - 166s 426ms/step - loss: 0.9184 - acc: 0.6727 - val_loss: 1.1719 - val_acc: 0.6089
Epoch 9/10
390/390 [==============================] - 165s 424ms/step - loss: 0.8749 - acc: 0.6892 - val_loss: 1.1652 - val_acc: 0.6136
Epoch 10/10
 27/390 [=>............................] - ETA: 2:21 - loss: 0.8338 - acc: 0.7034

390/390 [==============================] - 165s 424ms/step - loss: 0.8422 - acc: 0.7023 - val_loss: 1.1013 - val_acc: 0.6377


In [51]:
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.SGD(lr=0.1),
              metrics=['accuracy'])

train_datagen=ImageDataGenerator(rescale=1./255,
shear_range=0.2,
zoom_range=0.2,horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)
model.fit_generator(train_datagen.flow(x_train, y_train, batch_size=batch_size),
                    samples_per_epoch=len(x_train) , epochs=15,
                   validation_data=val_datagen.flow(x_test,y_test,batch_size=val_batch_size)
                   ,verbose=1)
# determine Loss function and Optimizer


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=15, validation_data=<keras.pre..., verbose=1, steps_per_epoch=390)`
  if sys.path[0] == '':


Epoch 1/15
390/390 [==============================] - 298s 764ms/step - loss: 0.8070 - acc: 0.7154 - val_loss: 0.8654 - val_acc: 0.7093
Epoch 2/15
280/390 [====================>.........] - ETA: 1:10 - loss: 0.7567 - acc: 0.7333

390/390 [==============================] - 272s 698ms/step - loss: 0.7502 - acc: 0.7359 - val_loss: 0.8907 - val_acc: 0.7004
Epoch 3/15
390/390 [==============================] - 272s 699ms/step - loss: 0.7049 - acc: 0.7543 - val_loss: 0.9146 - val_acc: 0.7020
Epoch 4/15
 17/390 [>.............................] - ETA: 3:58 - loss: 0.6616 - acc: 0.7734

390/390 [==============================] - 272s 698ms/step - loss: 0.6744 - acc: 0.7628 - val_loss: 0.9179 - val_acc: 0.6993
Epoch 5/15
320/390 [=======================>......] - ETA: 44s - loss: 0.6475 - acc: 0.7751

390/390 [==============================] - 273s 700ms/step - loss: 0.6441 - acc: 0.7766 - val_loss: 1.0036 - val_acc: 0.6769
Epoch 6/15
390/390 [==============================] - 273s 699ms/step - loss: 0.6254 - acc: 0.7809 - val_loss: 0.7173 - val_acc: 0.7498
Epoch 7/15
 26/390 [=>............................] - ETA: 3:52 - loss: 0.5912 - acc: 0.7861

...

390/390 [==============================] - 273s 700ms/step - loss: 0.5869 - acc: 0.7963 - val_loss: 0.9578 - val_acc: 0.6947
Epoch 9/15
390/390 [==============================] - 273s 700ms/step - loss: 0.5664 - acc: 0.8027 - val_loss: 0.8783 - val_acc: 0.7196
Epoch 10/15
 27/390 [=>............................] - ETA: 3:50 - loss: 0.5549 - acc: 0.8070

390/390 [==============================] - 273s 699ms/step - loss: 0.5528 - acc: 0.8076 - val_loss: 0.6815 - val_acc: 0.7668
Epoch 11/15
323/390 [=======================>......] - ETA: 42s - loss: 0.5415 - acc: 0.8106

390/390 [==============================] - 273s 700ms/step - loss: 0.5386 - acc: 0.8120 - val_loss: 0.6799 - val_acc: 0.7674
Epoch 12/15
390/390 [==============================] - 273s 699ms/step - loss: 0.5265 - acc: 0.8185 - val_loss: 0.6178 - val_acc: 0.7891
Epoch 13/15
 25/390 [>.............................] - ETA: 3:52 - loss: 0.5245 - acc: 0.8137

390/390 [==============================] - 273s 699ms/step - loss: 0.5165 - acc: 0.8186 - val_loss: 0.6150 - val_acc: 0.7872
Epoch 14/15
322/390 [=======================>......] - ETA: 43s - loss: 0.4961 - acc: 0.8290

390/390 [==============================] - 273s 700ms/step - loss: 0.4995 - acc: 0.8272 - val_loss: 0.7869 - val_acc: 0.7448
Epoch 15/15
390/390 [==============================] - 272s 697ms/step - loss: 0.4898 - acc: 0.8285 - val_loss: 0.8649 - val_acc: 0.7198


In [52]:
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.SGD(lr=0.01),
              metrics=['accuracy'])

train_datagen=ImageDataGenerator(rescale=1./255,
shear_range=0.2,
zoom_range=0.2,horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)
model.fit_generator(train_datagen.flow(x_train, y_train, batch_size=batch_size),
                    samples_per_epoch=len(x_train) , epochs=12,
                   validation_data=val_datagen.flow(x_test,y_test,batch_size=val_batch_size)
                   ,verbose=1)
# determine Loss function and Optimizer

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=12, validation_data=<keras.pre..., verbose=1, steps_per_epoch=390)`
  if sys.path[0] == '':


Epoch 1/12
390/390 [==============================] - 297s 761ms/step - loss: 0.4209 - acc: 0.8537 - val_loss: 0.5117 - val_acc: 0.8230
Epoch 2/12
280/390 [====================>.........] - ETA: 1:10 - loss: 0.4078 - acc: 0.8573

390/390 [==============================] - 273s 700ms/step - loss: 0.4046 - acc: 0.8586 - val_loss: 0.5079 - val_acc: 0.8253
Epoch 3/12
390/390 [==============================] - 273s 700ms/step - loss: 0.4004 - acc: 0.8612 - val_loss: 0.4831 - val_acc: 0.8319
Epoch 4/12
 17/390 [>.............................] - ETA: 4:00 - loss: 0.3761 - acc: 0.8667

390/390 [==============================] - 273s 700ms/step - loss: 0.3910 - acc: 0.8644 - val_loss: 0.4903 - val_acc: 0.8318
Epoch 5/12
320/390 [=======================>......] - ETA: 44s - loss: 0.3818 - acc: 0.8682

390/390 [==============================] - 273s 699ms/step - loss: 0.3833 - acc: 0.8675 - val_loss: 0.5009 - val_acc: 0.8308
Epoch 6/12
390/390 [==============================] - 272s 698ms/step - loss: 0.3845 - acc: 0.8666 - val_loss: 0.5090 - val_acc: 0.8277
Epoch 7/12
 26/390 [=>............................] - ETA: 3:51 - loss: 0.3924 - acc: 0.8642

390/390 [==============================] - 272s 698ms/step - loss: 0.3827 - acc: 0.8661 - val_loss: 0.4942 - val_acc: 0.8336
Epoch 8/12
324/390 [=======================>......] - ETA: 42s - loss: 0.3802 - acc: 0.8679

390/390 [==============================] - 272s 697ms/step - loss: 0.3796 - acc: 0.8688 - val_loss: 0.4900 - val_acc: 0.8325
Epoch 9/12
390/390 [==============================] - 273s 700ms/step - loss: 0.3763 - acc: 0.8697 - val_loss: 0.5261 - val_acc: 0.8240
Epoch 10/12
 27/390 [=>............................] - ETA: 3:52 - loss: 0.3817 - acc: 0.8689

390/390 [==============================] - 272s 699ms/step - loss: 0.3727 - acc: 0.8718 - val_loss: 0.4971 - val_acc: 0.8332
Epoch 11/12
323/390 [=======================>......] - ETA: 42s - loss: 0.3712 - acc: 0.8705

390/390 [==============================] - 273s 699ms/step - loss: 0.3698 - acc: 0.8708 - val_loss: 0.4963 - val_acc: 0.8348
Epoch 12/12
390/390 [==============================] - 273s 699ms/step - loss: 0.3711 - acc: 0.8724 - val_loss: 0.4729 - val_acc: 0.8392


In [53]:
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.SGD(lr=0.001),
              metrics=['accuracy'])

train_datagen=ImageDataGenerator(rescale=1./255,
shear_range=0.2,
zoom_range=0.2,horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)
model.fit_generator(train_datagen.flow(x_train, y_train, batch_size=batch_size),
                    samples_per_epoch=len(x_train) , epochs=7,
                   validation_data=val_datagen.flow(x_test,y_test,batch_size=val_batch_size)
                   ,verbose=1)
# determine Loss function and Optimizer

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=7, validation_data=<keras.pre..., verbose=1, steps_per_epoch=390)`
  if sys.path[0] == '':


Epoch 1/7
390/390 [==============================] - 296s 759ms/step - loss: 0.3622 - acc: 0.8752 - val_loss: 0.4785 - val_acc: 0.8383
Epoch 2/7
284/390 [====================>.........] - ETA: 1:07 - loss: 0.3580 - acc: 0.8748

390/390 [==============================] - 272s 697ms/step - loss: 0.3604 - acc: 0.8742 - val_loss: 0.4763 - val_acc: 0.8401
Epoch 3/7
390/390 [==============================] - 271s 694ms/step - loss: 0.3609 - acc: 0.8757 - val_loss: 0.4774 - val_acc: 0.8386
Epoch 4/7
 19/390 [>.............................] - ETA: 3:59 - loss: 0.3544 - acc: 0.8750

390/390 [==============================] - 271s 695ms/step - loss: 0.3614 - acc: 0.8752 - val_loss: 0.4785 - val_acc: 0.8386
Epoch 5/7
323/390 [=======================>......] - ETA: 42s - loss: 0.3611 - acc: 0.8731

390/390 [==============================] - 271s 696ms/step - loss: 0.3595 - acc: 0.8744 - val_loss: 0.4798 - val_acc: 0.8385
Epoch 6/7
390/390 [==============================] - 272s 698ms/step - loss: 0.3565 - acc: 0.8764 - val_loss: 0.4815 - val_acc: 0.8368
Epoch 7/7
 28/390 [=>............................] - ETA: 3:50 - loss: 0.3587 - acc: 0.8806

390/390 [==============================] - 272s 697ms/step - loss: 0.3589 - acc: 0.8753 - val_loss: 0.4769 - val_acc: 0.8388


In [54]:
model.evaluate(x_test/255.0,y_test,verbose=1)

10000/10000 [==============================] - 29s 3ms/step


[0.47685202322006226, 0.8388]

In [61]:
model.evaluate(x_test_resized/255.0,y_test,verbose=1)


10000/10000 [==============================] - 18s 2ms/step


[0.7252554464817047, 0.7658]

In [62]:

x_test_expanded=np.array([scipy.misc.imresize(img,(64,64)) for img in x_test])


/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:482: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if issubdtype(ts, int):
/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:485: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif issubdtype(type(size), float):


In [63]:
model.evaluate(x_test_expanded/255.0,y_test,verbose=1)

10000/10000 [==============================] - 76s 8ms/step


[2.693086283111572, 0.2915]

In [0]:
# Test the model
score = model.evaluate(x_test/255.0, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 52s 5ms/step
Test loss: 0.4965174989700317
Test accuracy: 0.8409


In [0]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model_improved.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model_improved.h5')